In [1]:
import pandas as pd
import numpy as np
import os
train_path1 = r"../data/train_fea/20200801.csv"
train_path2 = r"../data/train_fea/20200802.csv"
test_path = r"../data/test_fea/20200901.csv"
submission_path = r"../sample_submission.csv"
# test_link_data = np.load('./data/train_fea/20200801_links.npz')
# test_link_data = test_link_data['data']
# print(test_link_data['data'])
submission = pd.read_csv(submission_path)
train_data1 = pd.read_csv(train_path1)
train_data1['date'] = 1
train_data2 = pd.read_csv(train_path2)
train_data2['date'] = 2

train_data = pd.concat([train_data1,train_data2])
test_data =  pd.read_csv(test_path)
test_data['date'] = 3
data = pd.concat([train_data,test_data])

In [3]:
os.system('pip install ipywidgets')

0

In [4]:
os.system("pip install gensim==3.8.3")

0

In [5]:
os.system("pip install node2vec")

0

In [6]:
os.system("pip install deepwalk")

0

In [7]:
from ipywidgets import IntProgress

In [7]:
path = "./"

In [2]:
import networkx as nx
import node2vec as n2v
from node2vec import Node2Vec

In [2]:
import pandas as pd
import numpy as np
import gc
import pickle
import time
import datetime
import warnings 
warnings.filterwarnings('ignore')

In [5]:
df_new = data
df_new.head()

,order_id,ata,distance,simple_eta,driver_id,slice_id,weather,hightemp,lowtemp,true_t,wight_t,cross_t,status_t,link_num,cross_num,date
0,5966280,531,2923.0000,495,75457,153,5,28,26,368.0639,441.57001,127.0,129,65,5,1
1,6099942,590,3212.2780,559,19210,157,5,28,26,475.7070,580.64778,83.0,135,63,4,1
2,2792816,559,7868.4474,626,38822,36,5,28,26,596.1600,709.76583,30.0,115,62,1,1
3,5780075,677,4976.3791,446,51627,76,5,28,26,414.2489,497.09868,32.0,140,70,1,1
4,7808558,968,4985.9186,872,39965,52,5,28,26,754.9907,878.61882,117.0,132,75,5,1


In [4]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [6]:
temp = df_new[['driver_id','slice_id']].groupby(['driver_id','slice_id'],as_index=False)['driver_id'].agg({'count'}).reset_index()
temp.head()

,driver_id,slice_id,count
0,0,15,1
1,0,30,1
2,0,37,1
3,0,55,1
4,0,58,1


In [9]:
lbl1,lbl2 = LabelEncoder(),LabelEncoder()
temp['slice_id'] = lbl1.fit_transform(temp['slice_id'].map(str))
temp['driver_id'] = lbl2.fit_transform(temp['driver_id']) + (temp['slice_id'].max()+1)

In [10]:
temp.to_csv('./cache/driver_slice_deepwalk.csv',index=False,header=False,sep=' ')

In [11]:
from gensim.models.word2vec import Vocab

In [12]:
%%time
import os
os.system('deepwalk --input ./cache/driver_slice_deepwalk.csv --format edgelist \
--output ./cache/driver_slice_deepwalk.emb --workers 40')

Wall time: 10min 19s


0

In [13]:
npy = np.loadtxt('./cache/driver_slice_deepwalk.emb', delimiter=' ',skiprows=1)

In [14]:
kfc = pd.DataFrame()
kfc['driver_id'] = npy[:, 0]
for i in range(1, 33):
    #print (i)
    kfc['driver_slice_deepwalk_' + str(i)] = npy[:, i]

In [15]:
kfc = kfc[~kfc['driver_id'].isin(temp['slice_id'].unique())]

In [16]:
kfc['driver_id'] = kfc['driver_id'] - (temp['slice_id'].max()+1)

kfc['driver_id'] = kfc['driver_id'].astype(int)


In [17]:
kfc['driver_id'] = lbl2.inverse_transform(kfc['driver_id'])

kfc.to_pickle('./cache/kfc_driver_slice_new_04.pkl')

In [18]:
import networkx as nx
import node2vec as n2v
from node2vec import Node2Vec

In [19]:
G = nx.DiGraph()
G.add_weighted_edges_from(temp[['driver_id','slice_id','count']].values)

In [20]:
%%time
node2vec = Node2Vec(G, dimensions=32,walk_length=20,num_walks=180,workers=32)#walk_length32

Computing transition probabilities:   0%|          | 0/55932 [00:00<?, ?it/s]

Wall time: 34min 11s


In [21]:
%%time
model = node2vec.fit(window=5,min_count=1,batch_words=8)

Wall time: 31min 45s


In [22]:
model.wv.save_word2vec_format(path+'cache/kfc_driver_slice_new_04.bin')

In [24]:
npy = np.loadtxt(path+'cache/kfc_driver_slice_new_04.bin', delimiter=' ',skiprows=1)
n2v = pd.DataFrame()
n2v['driver_id'] = npy[:, 0]
for i in range(1, 33):
    #print (i)
    n2v['driver_slice_node2vec_' + str(i)] = npy[:, i]
    
n2v = n2v[~n2v['driver_id'].isin(temp['slice_id'].unique())]
n2v['driver_id'] = n2v['driver_id'] - (temp['slice_id'].max()+1)
n2v['driver_id'] = n2v['driver_id'].astype(int)
n2v['driver_id'] = lbl2.inverse_transform(n2v['driver_id'])

In [25]:
n2v.head()

,driver_id,driver_slice_node2vec_1,driver_slice_node2vec_2,driver_slice_node2vec_3,driver_slice_node2vec_4,driver_slice_node2vec_5,driver_slice_node2vec_6,driver_slice_node2vec_7,driver_slice_node2vec_8,driver_slice_node2vec_9,...,driver_slice_node2vec_23,driver_slice_node2vec_24,driver_slice_node2vec_25,driver_slice_node2vec_26,driver_slice_node2vec_27,driver_slice_node2vec_28,driver_slice_node2vec_29,driver_slice_node2vec_30,driver_slice_node2vec_31,driver_slice_node2vec_32
288,32841,0.251528,0.204280,-0.211265,0.149438,-0.582420,-1.030134,-0.170559,-0.178498,-0.050722,...,-0.396895,-0.366272,-0.801018,0.024581,-0.180310,-0.657225,0.564511,-1.073502,-1.041257,0.530785
289,5110,0.162684,-0.298046,0.384109,0.165838,-0.662947,0.604299,-0.367587,-0.366502,-0.686390,...,-0.082310,-0.504325,-0.384169,-0.421684,-0.701152,-0.805651,0.190792,-0.673654,0.050129,-0.006333
290,23497,0.285328,0.308566,0.406302,0.522508,-0.124148,-0.062725,-0.301508,-0.639524,0.208179,...,-0.048451,0.508452,-0.618720,0.134620,-0.916013,-0.638024,0.096409,-0.732910,-0.439665,0.012675
291,29482,0.743864,-0.305968,0.655279,0.558280,-0.121892,-0.479592,-0.362802,-0.167520,-0.664655,...,-0.639096,-0.647118,-1.051379,-0.355336,-0.411868,-0.779836,-0.003785,-0.475568,0.146834,-0.469208
292,999,0.978022,0.159972,0.076900,1.066224,0.465277,0.113227,-0.402245,-0.334823,0.453720,...,-0.473628,0.172872,0.062855,-0.183283,-0.795612,-0.804481,0.299493,-0.072041,-0.411775,0.377886


In [26]:
n2v.to_csv('./cache/kfc_driver_slice_new_n2v.csv',index=0)